In [15]:
# Import Statements
from pulp import LpProblem, LpVariable, lpSum
import pandas as pd

In [16]:
# Reading in the CSV file from Kaggle (Credits to Paola Mazza) into a Pandas Data Frame
players_df = pd.read_csv("players.csv")

In [19]:
def optimize_team(players_df, max_cost, max_keepers, max_defenders, max_midfielders, max_forwards):
    # Create a linear programming problem
    prob = LpProblem("TeamOptimization", LpMaximize)

    # Create binary decision variables for each player
    players_df['selected'] = LpVariable.dicts("Player", players_df.index, cat="Binary")

    # Objective function: Maximize total points
    prob += lpSum(players_df['total_points'] * players_df['selected'])

    # Cost constraint: Total cost should be less than max_cost
    prob += lpSum(players_df['now_cost'] * players_df['selected']) <= max_cost

    # Position constraints: Limit the number of players from each role
    prob += lpSum(players_df['selected'][players_df['position'] == 'GKP']) <= max_keepers
    prob += lpSum(players_df['selected'][players_df['position'] == 'DEF']) <= max_defenders
    prob += lpSum(players_df['selected'][players_df['position'] == 'MID']) <= max_midfielders
    prob += lpSum(players_df['selected'][players_df['position'] == 'FWD']) <= max_forwards

    # Solve the problem
    prob.solve()

    # Extract the selected players
    selected_players = players_df.loc[players_df['selected'].apply(lambda x: x.varValue) == 1]

    return selected_players

In [20]:
# Set the maximum budget and position constraints
max_budget = 1000
max_keepers = 2
max_defenders = 5
max_midfielders = 5
max_forwards = 3

# Call the optimize_team function
selected_team = optimize_team(players_df, max_budget, max_keepers, max_defenders, max_midfielders, max_forwards)

# Display the selected team
print("Selected Team:")
print(selected_team)

Selected Team:
      id                          name  now_cost position            team  \
18    20                William Saliba        55      DEF         Arsenal   
25    31           Oleksandr Zinchenko        52      DEF         Arsenal   
45    43  Douglas Luiz Soares de Paulo        55      MID     Aston Villa   
61    60                 Ollie Watkins        88      FWD     Aston Villa   
95    85               Dominic Solanke        69      FWD     Bournemouth   
281  220              Joachim Andersen        49      DEF  Crystal Palace   
330  263               Jordan Pickford        44      GKP         Everton   
332  265               James Tarkowski        45      DEF         Everton   
400  308                 Mohamed Salah       133      MID       Liverpool   
463  343                Julián Álvarez        68      FWD        Man City   
522  597                   André Onana        48      GKP         Man Utd   
544  412                Anthony Gordon        62      MID    